In [3]:
import numpy as np
import pandas as pd
import gdreg
import time
import matplotlib.pyplot as plt

# autoreload
%load_ext autoreload
%autoreload 2

In [4]:
DATA_PATH='/n/groups/price/martin/data_GDREG/'
OUT_PATH = '/n/groups/price/martin/data_GDREG/imp_geno'

In [ ]:
# # Check MHC 
# temp_df = pd.read_csv('/n/groups/price/martin/WES_analysis/toy_1K/weights_hm3_no_hla/weights.6.l2.ldscore.gz',
#                       sep='\t')
# plt.figure(figsize=[8,5])
# plt.scatter(temp_df['BP'], [1]*temp_df.shape[0])
# plt.xlim([2.4e7,3.5e7])
# plt.show()

# print(temp_df['BP'][temp_df['BP']<3e7].max())
# print(temp_df['BP'][temp_df['BP']>3e7].min())

### 337K sample list 

In [7]:
df_unrelated = pd.read_csv('/n/groups/price/UKBiobank/sampleQC/samples_337K.txt', sep=' ', header=None)
df_unrelated.columns = ['FID', 'IID']
df_unrelated.index = ['%s_%s'%(x,y) for x,y in zip(df_unrelated['FID'], df_unrelated['IID'])]
print('df_unrelated: %d'%len(df_unrelated.index))

df_remove = pd.read_csv(
    '/n/groups/price/UKBiobank/download_500K/w14048_CURRENT.FID_IID.txt', sep=' ', header=None
)
df_remove.columns = ['FID', 'IID']
df_remove.index = ['%s_%s'%(x,y) for x,y in zip(df_remove['FID'], df_remove['IID'])]
print('df_remove: %d'%len(df_remove.index))

# Subsample ID_list
legit_id_list = sorted(set(df_unrelated.index) - set(df_remove.index))
print('legit_id_list: %d'%len(legit_id_list))

temp_set = set(legit_id_list)
df_sample = df_unrelated.loc[[x in temp_set for x in df_unrelated.index]]
df_sample.to_csv(OUT_PATH+'/unrelated_337K.txt', sep=' ', header=False, index=False)
df_sample.iloc[:2].to_csv(OUT_PATH+'/unrelated_337K.small.txt', sep=' ', header=False, index=False)
print('df_sample: %d'%len(df_sample.index))

df_unrelated: 337545
df_remove: 242
legit_id_list: 337426
df_sample: 337426


### MAF>0.1% and INFO>0.6 SNP list

In [19]:
n_snp = 0
for CHR in range(1,23):
# for CHR in [21]:
    temp_df = pd.read_csv(
        "/n/groups/price/UKBiobank/download_500K/ukb_mfi_chr%d_v3.txt.gz"%CHR, sep='\t', header=None
    )
    
    ind_select = (temp_df[5]>=0.001) & (temp_df[7]>=0.6)
    snp_list = list(temp_df[1].values[ind_select])
    print("CHR%d, n_snp=%d/%d (%d with MAF>0.001)"%(
        CHR, len(snp_list), temp_df.shape[0], (temp_df[5]>=0.001).sum() 
    ))
    n_snp += len(snp_list)

    with open("/n/groups/price/martin/data_GDREG/imp_geno/snp_list_chr%d.MAF_001_INFO_06.txt"%CHR, "w") as f:
        for snp in snp_list:
            f.write("%s\n"%snp)
            
print("Total %d SNPs" % n_snp)

CHR1, n_snp=1444017/7402791 (1542314 with MAF>0.001)
CHR2, n_snp=1560656/8129063 (1649892 with MAF>0.001)
CHR3, n_snp=1315193/6696680 (1382954 with MAF>0.001)
CHR4, n_snp=1327041/6555871 (1400212 with MAF>0.001)
CHR5, n_snp=1189519/6070641 (1248942 with MAF>0.001)
CHR6, n_snp=1196493/5751712 (1257590 with MAF>0.001)
CHR7, n_snp=1078277/5405524 (1141349 with MAF>0.001)
CHR8, n_snp=1029189/5282223 (1085553 with MAF>0.001)
CHR9, n_snp=794311/4066774 (849684 with MAF>0.001)
CHR10, n_snp=924158/4562904 (974698 with MAF>0.001)
CHR11, n_snp=907848/4628348 (960498 with MAF>0.001)
CHR12, n_snp=879579/4431052 (932167 with MAF>0.001)
CHR13, n_snp=661335/3270217 (696124 with MAF>0.001)
CHR14, n_snp=597763/3037521 (635841 with MAF>0.001)
CHR15, n_snp=533036/2767971 (573469 with MAF>0.001)
CHR16, n_snp=582568/3089612 (622814 with MAF>0.001)
CHR17, n_snp=512905/2660711 (546124 with MAF>0.001)
CHR18, n_snp=521824/2599579 (548212 with MAF>0.001)
CHR19, n_snp=427378/2087017 (454439 with MAF>0.001)
CHR20

CHR21, n_snp=250508/1261158


In [9]:
dic_data = gdreg.util.read_pgen("/n/groups/price/martin/data_GDREG/imp_geno/ukb_imp_chr21_v3")

In [10]:
temp_mat = gdreg.util.read_geno(dic_data['pgen'], 0, 50, n_sample=None, n_snp=None)

In [11]:
temp_mat.mean(axis=1)

array([-1.39289800e-02, -2.85040275e-02, -1.39556525e-02, -1.39556525e-02,
       -1.14928903e-02, -3.94812492e-02, -3.52373557e-03,  2.34125408e-04,
       -2.32169424e-02, -1.39141619e-02, -6.80741852e-03, -9.53423862e-02,
       -7.33494159e-03, -1.08764588e-02, -1.23760469e-02, -2.33503050e-02,
       -2.69955487e-02, -2.69955487e-02, -3.23715422e-02, -6.80741852e-03,
       -1.11135479e-02, -2.33503050e-02, -1.67029215e-02, -1.21069509e-01,
       -2.85040275e-02, -2.85040275e-02,  2.60797923e-04,  2.60797923e-04,
       -4.77141655e-03, -3.11090432e-02, -7.28159656e-03, -2.89544967e-02,
       -1.21359943e-02, -7.25818402e-02,  2.60797923e-04, -2.64294986e-02,
       -6.48912650e-02, -4.45709578e-01, -5.21228358e-01, -3.79105345e-02,
       -4.84989301e-01, -4.93560069e-02, -4.88924979e-01, -3.01405345e-01,
       -5.39170070e-01, -5.01446243e-01, -5.06505130e-01, -2.81812901e-01,
       -4.27604867e-01, -5.00915756e-01])